# Environment dependencies

In [16]:
# !conda create -n name python=3.10
# !conda install pytorch torchvision torchaudio pytorch-cuda=11.7 -c pytorch -c nvidia
# !conda install matplotlib numpy scipy scikit-learn jupyter
# !conda install -c conda-forge scikit-optimize

In [17]:
import numpy as np
import os
import time

import torch
from torch import nn
from torch.nn import functional as F
import deepxde.deepxde as dde
from deepxde.deepxde.nn.pytorch.deeponet import DeepONetCartesianProd

from utils.func import *
from utils.loss import dataLoss, pinnLoss
from utils.op import *
from utils.pdes import diffusion_reaction_nointep as diffusion_reaction
from utils.logger import getLogger
from dataset import parallel_solver
from dataset.ADRSolver import diffusion_reaction_solver
pde = lambda x, y: diffusion_reaction(x, y, D = 0.01, k = 0.01)

import matplotlib.pyplot as plt

In [18]:
# Config
testing = True      # testing the code or not
name = "Example"    # name of the model
info = "This code is a basic example of the DeepONet using physics-informed active learning."
testing_data = "dataset/DF_1000test_ls0.1_101x101.npz"

if testing:
    n_init = 20 # number of v(x) use to pre-train the model
    n_0 = 100 # number of v(x) would be selected from the training data
    n_1 = 20 # number of v(x) in n0 would be used to train the model
    n_2 = 200 # total v(x) would be used to train the model

    batchsize = 1 # batchsize for training, 5000 for testing the code. Defaults to 20000.
    iteration = 1000 # iteration for training, 5000 for testing the code. Defaults to 30000.
    shuf_dataset = True # shuffle the dataset or not

    lr = 1e-3 # learning rate
    decay_step = iteration // 5 # decay step, using inverse time decay 
    decay_rate = 0.5
else:
    n_init = 20 # number of v(x) use to pre-train the model
    n_0 = 100 # number of v(x) would be selected from the training data
    n_1 = 20 # number of v(x) in n0 would be used to train the model
    n_2 = 200 # total v(x) would be used to train the model

    batchsize = 1 # batchsize for training, 5000 for testing the code. Defaults to 20000.
    iteration = 20000 # iteration for training, 5000 for testing the code. Defaults to 30000.
    shuf_dataset = True # shuffle the dataset or not

    lr = 1e-3 # learning rate
    decay_step = iteration // 5 # decay step, using inverse time decay 
    decay_rate = 0.5

dir_name = f"{name}-{time.strftime('%m%d-%H%M%S', time.localtime())}" if not testing else "test"
os.makedirs(f"model/{dir_name}", exist_ok = True)

logger = getLogger(f"model/{dir_name}/train.log")
logger.info(info)
logger.info(f"Using {torch.cuda.device_count()} GPU(s). The name of devices: {torch.cuda.get_device_name()}")
logger.info(f"Training hyperparameters:")
logger.info(f"batchsize: {batchsize}, iteration: {iteration}, lr = {lr}, decay_step = {decay_step}, decay_rate = {decay_rate}, initial data: {n_init}, testing number: {n_0}, selecting number: {n_1}, total data: {n_2}.")


Main - 13:17:16 - INFO: This code is a basic example of the DeepONet using physics-informed active learning.
Main - 13:17:16 - INFO: Using 1 GPU(s). The name of devices: NVIDIA GeForce GTX 1650 with Max-Q Design
Main - 13:17:16 - INFO: Training hyperparameters:
Main - 13:17:16 - INFO: batchsize: 1, iteration: 1000, lr = 0.001, decay_step = 200, decay_rate = 0.5, initial data: 20, testing number: 100, selecting number: 20, total data: 200.


# Network understanding
Input function $u$ is the unknown function given by the sensor. Every sensor has a different location, so the input function can be approximated by these sensors. Let the $i$-th sensor be located at $\bm{x_i}$. Then the input function is $u(\bm{x_i})$.

Therefore, if we have 100 sensors, we have 100 values of $u_i = u(\bm{x_i})$. Then the Branch net would use these 100 $u_i$ as inputs and output given number of $b_i$.

For the Trunk net, since we got the $\bm{x_i}$ for every sensor, we could make use of these $\bm{x_i}$ to train the Trunk net. The Trunk net takes $\bm{x_i}$ as input and output $t_i$.

Finally, the deepONet output is:
$$ \sum_i{b_it_i} + b_0. $$

# Data for this problem
The goal of us is to solve $u(x,t)$ by the following equation and given $v(x)$.
$$ \frac{\partial u}{\partial t} = D \frac{\partial^2 u}{\partial x^2} + ku^2 + v(x), \quad x,t \in [0,1]$$

Here, we sample the points $\bm{x} = (x,t)$ uniformly in $[0,1] \times [0,1]$. And we provide 1000 different $v(x)$, each one has 101 uniformly sampled points in $[0,1]$. For $u(x,t)$, we have 10201 uniformly sampled points in $[0,1] \times [0,1]$. 

In [19]:
space = dde.data.GRF(1.0, length_scale = 0.1, N= 1000, interp="cubic")
vxs = space.eval_batch(space.random(20), np.linspace(0, 1, 101)[:, None])
uxts = parallel_solver(diffusion_reaction_solver, vxs)
uxts, grid = np.asarray([utx[1] for utx in uxts]), uxts[0][0]

train_vxs = vxs.astype(np.float32)
train_uxts = uxts.reshape(uxts.shape[0], -1).astype(np.float32)
train_grid = grid.reshape(-1, grid.shape[-1]).astype(np.float32)
logger.info(f"Training data: Random")
logger.info(f"vx shape {train_vxs.shape}, uxt shape {train_uxts.shape}, grid shape {train_grid.shape}.")

test_data = np.load(testing_data)
vxs, uxts, grid = test_data["vxs"], test_data["uxts"], test_data["xt"]

test_vxs = vxs.astype(np.float32)[:100]
test_uxts = uxts.reshape(uxts.shape[0], -1).astype(np.float32)[:100]
test_grid = grid.reshape(-1, grid.shape[-1]).astype(np.float32)
logger.info(f"Testing data: {testing_data}.")
logger.info(f"vx shape {test_vxs.shape}, uxt shape {test_uxts.shape}, grid shape {test_grid.shape}.")

Main - 13:17:17 - INFO: Training data: Random
Main - 13:17:17 - INFO: vx shape (20, 101), uxt shape (20, 10201), grid shape (10201, 2).
Main - 13:17:17 - INFO: Testing data: dataset/DF_1000test_ls0.1_101x101.npz.
Main - 13:17:17 - INFO: vx shape (100, 101), uxt shape (100, 10201), grid shape (10201, 2).


In [22]:
data = PDETripleCartesianProd(X_train=(train_vxs, train_grid), y_train=train_uxts, X_test=(test_vxs, test_grid), y_test=test_uxts)

net = DeepONetCartesianProd(
        layer_sizes_branch= [101, 100, 100],
        layer_sizes_trunk= [2, 100, 100, 100],
        activation= {"branch": F.gelu, "trunk": F.gelu},
        kernel_initializer= "Glorot normal")

net.apply_output_transform(dirichlet)

checker = dde.callbacks.ModelCheckpoint(f"model/{dir_name}/{name}_", save_better_only=False, period=10000)

model = dde.Model(data, net)
model.compile("adam", loss = [dataLoss(), ], lr=lr, metrics=[], decay = (torch.optim.lr_scheduler.LambdaLR, [], {"lr_lambda": lambda step: 1 / (1 + decay_rate * (step / decay_step))}))
#pinnLoss(diffusion_reaction)

Compiling model...
'compile' took 0.000386 s



In [23]:
# First train without any techniques
loss_history, train_state = model.train(iterations = 20000,
                                        callbacks = [checker],
                                        batch_size = batchsize,
                                        test_batch_size = 10,
                                        display_every = 5000,
                                        )

Training model...

0         [6.02e-01]    [3.28e-01]    []  
5000      [5.54e-02]    [7.13e-02]    []  
10000     [8.26e-03]    [6.03e-02]    []  
15000     [1.73e-02]    [6.02e-02]    []  
20000     [5.70e-03]    [6.10e-02]    []  

Best model at step 20000:
  train loss: 5.70e-03
  test loss: 6.10e-02
  test metric: []

'train' took 162.697684 s



In [24]:
def test_new_vxs(model, vx_number: int = 10, grid: np.ndarray = grid):
    space = dde.data.GRF(1.0, length_scale = 0.1, N= 1000, interp="cubic")
    vxs = space.eval_batch(space.random(10), np.linspace(0, 1, 101)[:, None])
    vxs = torch.from_numpy(vxs)
    grid = torch.from_numpy(grid)
    results = model.predict((vxs, grid), pinnLoss(diffusion_reaction, func = lambda x, y: (x -y).abs()))
    return vxs, torch.from_numpy(results)

In [25]:
while len(train_vxs) < n_2:
    vxs, results = [], []
    for i in range(n_0 // 10):
        vx, result = test_new_vxs(model, vx_number=10, grid=train_grid)
        vxs.append(vx)
        results.append(result)
    vxs = torch.cat(vxs)
    results = torch.cat(results)
    results = results.mean(axis=1)[..., 0]
    indices = torch.topk(results, n_1)[1]
    vxs = vxs[indices]
    vxs = vxs.numpy()
    uxts = parallel_solver(diffusion_reaction_solver, vxs)
    uxts, grid = np.asarray([utx[1] for utx in uxts]), uxts[0][0]

    train_vxs = np.concatenate([train_vxs, vxs], 0).astype(np.float32)
    train_uxts = np.concatenate(
        [train_uxts, uxts.reshape(uxts.shape[0], -1)], 0).astype(np.float32)

    data = PDETripleCartesianProd(X_train=(train_vxs, train_grid), y_train=train_uxts, X_test=(
        test_vxs, test_grid), y_test=test_uxts)

    model = dde.Model(data, net)
    model.compile("adam", loss=[dataLoss(),], lr=lr, metrics=[], decay=(torch.optim.lr_scheduler.LambdaLR, [
    ], {"lr_lambda": lambda step: 1 / (1 + decay_rate * (step / decay_step))}))
    # pinnLoss(diffusion_reaction)
    loss_history, train_state = model.train(iterations=20000,
                                            callbacks=[checker],
                                            batch_size=batchsize,
                                            test_batch_size=10,
                                            display_every=5000,
                                            )

Compiling model...
'compile' took 0.001362 s

Training model...

Step      Train loss    Test loss     Test metric
0         [4.65e-03]    [6.10e-02]    []  
5000      [1.88e-03]    [3.68e-02]    []  
10000     [8.52e-03]    [3.87e-02]    []  
15000     [6.65e-03]    [3.84e-02]    []  
20000     [1.66e-03]    [3.85e-02]    []  

Best model at step 20000:
  train loss: 1.66e-03
  test loss: 3.85e-02
  test metric: []

'train' took 142.073638 s

Compiling model...
'compile' took 0.000603 s

Training model...

Step      Train loss    Test loss     Test metric
0         [7.27e-03]    [3.85e-02]    []  
5000      [1.06e-02]    [2.04e-02]    []  
10000     [2.67e-03]    [1.92e-02]    []  
15000     [5.81e-03]    [1.89e-02]    []  
20000     [4.72e-03]    [1.87e-02]    []  

Best model at step 10000:
  train loss: 2.67e-03
  test loss: 1.92e-02
  test metric: []

'train' took 149.845083 s

Compiling model...
'compile' took 0.000619 s

Training model...

Step      Train loss    Test loss     T

In [ ]:
data = dde.data.Triple(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test)
model = dde.Model(data, net)
model.compile("adam", lr=lr)
model.restore("model/model0.ckpt-23000.pt", verbose=1)

index = 1

v_x = test_data["X_test0"][index] # 101
u_gt = test_data["y_test"][index]

X = test_data["X_test1"] # 10201, 2

sensor_value = v_x[None, :].repeat(10201, axis=0) # 10201, 101

u_pd = model.predict((sensor_value, X))[:,0] # 10201, 1
fig, (ax1, ax2,ax3) = plt.subplots(1, 3)
fig.suptitle("$u(x,t)$")
fig.set_size_inches(4 * 3,  4)
ax1.set_title(f"{index}_gt")
ax1.set_xlim([0, 1])
ax1.set_ylim([0, 1])
ax1.set_aspect('equal')
ax1.set_xlabel("$x$")
ax1.set_ylabel("$t$")
scatter = ax1.scatter(X[:,0], X[:, 1], c=u_gt)
colorbar = plt.colorbar(scatter, ax = ax1, shrink=0.7)

ax2.set_title(f"{index}_pd")
ax2.set_xlim([0, 1])
ax2.set_ylim([0, 1])
ax2.set_aspect('equal')
ax2.set_xlabel("$x$")
ax2.set_ylabel("$t$")
scatter = ax2.scatter(X[:,0], X[:, 1], c=u_pd)
colorbar = plt.colorbar(scatter, ax = ax2, shrink=0.7)

ax3.set_title(f"{index}_delta")
ax3.set_xlim([0, 1])
ax3.set_ylim([0, 1])
ax3.set_aspect('equal')
ax3.set_xlabel("$x$")
ax3.set_ylabel("$t$")
scatter = ax3.scatter(X[:,0], X[:, 1], c=u_pd - u_gt)
colorbar = plt.colorbar(scatter, ax = ax3, shrink=0.7)

plt.tight_layout()
plt.show()